# Poker Round Simulator

This is a clean version of the poker simulator that works reliably in JupyterLab.

**Features:**
1. Traditional Poker Hands with joker support
2. Exotic "Illegal" Hands like Rainbow Straight and Flush Five
3. Mod Cards that modify gameplay
4. Character Classes with unique abilities
5. Multi-Round Tournament system

Let's start by setting up the core components!

In [33]:
# Import all necessary components
# pylint: disable=import-error
from pydantic import BaseModel, Field
from typing import List, Optional, Dict, Tuple, Literal, Set, cast
import random
import enum
from itertools import combinations

print("Welcome to the Poker Round Simulator!")
print("This is a clean version that works reliably in JupyterLab.")

Welcome to the Poker Round Simulator!
This is a clean version that works reliably in JupyterLab.


In [34]:
# Define core data structures with full type safety

DEFAULT_STARTING_CHIPS = 50

class Rarity(str, enum.Enum):
    Common = "Common"
    Uncommon = "Uncommon"
    Rare = "Rare"
    Epic = "Epic"

class CharacterClass(str, enum.Enum):
    Warrior = "Warrior"
    Rogue = "Rogue"
    Mage = "Mage"
    Cleric = "Cleric"
    Bard = "Bard"
    Wizard = "Wizard"
    Jester = "Jester"
    Pauper = "Pauper"
    Knight = "Knight"

class ModCardEffects(str, enum.Enum):
    Draw1 = "Draw1"
    Draw2 = "Draw2"
    Discard1 = "Discard1"
    Discard2 = "Discard2"
    Steal1 = "Steal1"
    Steal2 = "Steal2"
    SneakySwap = "SneakySwap"
    RoyalDividend = "RoyalDividend"
    GainChips = "GainChips"

class ModCardType(str, enum.Enum):
    Player = "player"
    Showdown = "showdown"
    Payout = "payout"
    Global = "global"

class ModCardTypeFlavor(str, enum.Enum):
    Pawn = "Pawn"
    Knight = "Knight"
    Rook = "Rook"
    Queen = "Queen"

# Card types
CardValue = str  # Format: "{rank}{suit}" or "JOKER"

# Hand types
HandType = Literal[
    "High Card", "One Pair", "Two Pair", "Three of a Kind", "Straight", 
    "Flush", "Full House", "Four of a Kind", "Straight Flush", "Royal Flush",
    "Flush Four", "Sandwich Hand", "Odd Straight", "Even Straight", 
    "Skipping Straight", "Rainbow Straight", "Flush House", "Five of a Kind", "Flush Five"
]

class ModCard(BaseModel):
    name: str
    description: str
    effect: ModCardEffects
    type: ModCardType
    flavorType: ModCardTypeFlavor
    rarity: Rarity
    burnOnUse: Optional[bool] = False

class PlayerCharacterCard(BaseModel):
    name: str
    char_class: CharacterClass
    deck_modifier: Optional[str] = None
    ability: str
    starting_chips: int = DEFAULT_STARTING_CHIPS
    description: Optional[str] = None

print("Core data structures defined with type safety!")
print(f"Available character classes: {list(CharacterClass)}")
print(f"Available mod effects: {list(ModCardEffects)[:5]}...")

Core data structures defined with type safety!
Available character classes: [<CharacterClass.Warrior: 'Warrior'>, <CharacterClass.Rogue: 'Rogue'>, <CharacterClass.Mage: 'Mage'>, <CharacterClass.Cleric: 'Cleric'>, <CharacterClass.Bard: 'Bard'>, <CharacterClass.Wizard: 'Wizard'>, <CharacterClass.Jester: 'Jester'>, <CharacterClass.Pauper: 'Pauper'>, <CharacterClass.Knight: 'Knight'>]
Available mod effects: [<ModCardEffects.Draw1: 'Draw1'>, <ModCardEffects.Draw2: 'Draw2'>, <ModCardEffects.Discard1: 'Discard1'>, <ModCardEffects.Discard2: 'Discard2'>, <ModCardEffects.Steal1: 'Steal1'>]...


In [35]:
# Create game decks

def create_real_deck() -> List[CardValue]:
    """Create a standard 52-card deck + 2 jokers."""
    ranks = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A"]
    suits = ["S", "H", "D", "C"]  # Using letters instead of symbols to avoid encoding issues
    
    deck = []
    for rank in ranks:
        for suit in suits:
            deck.append(f"{rank}{suit}")
    
    # Add jokers
    deck.extend(["JOKER", "JOKER"])
    return deck

# Create game data
real_deck = create_real_deck()

mod_deck = [
    ModCard(
        name="Extra Draw", 
        description="Draw 1 additional card", 
        effect=ModCardEffects.Draw1, 
        type=ModCardType.Player, 
        flavorType=ModCardTypeFlavor.Pawn, 
        rarity=Rarity.Common
    ),
    ModCard(
        name="Sneaky Swap", 
        description="Swap a card with opponent", 
        effect=ModCardEffects.SneakySwap, 
        type=ModCardType.Player, 
        flavorType=ModCardTypeFlavor.Pawn, 
        rarity=Rarity.Uncommon
    ),
    ModCard(
        name="Royal Dividend", 
        description="Gain +3 chips", 
        effect=ModCardEffects.RoyalDividend, 
        type=ModCardType.Payout, 
        flavorType=ModCardTypeFlavor.Queen, 
        rarity=Rarity.Uncommon
    )
]

players = [
    PlayerCharacterCard(
        name="Scruffy McMuffins", 
        char_class=CharacterClass.Rogue, 
        deck_modifier="Normal", 
        ability="Steal 1 opponent card 2x per round"
    ),
    PlayerCharacterCard(
        name="Patches", 
        char_class=CharacterClass.Jester, 
        deck_modifier="Remove face cards", 
        ability="Blind money refunded"
    ),
    PlayerCharacterCard(
        name="Sir Whiskers", 
        char_class=CharacterClass.Warrior, 
        deck_modifier="Normal", 
        ability="Reveal opponent card once per round"
    )
]

print(f"Real Deck: {len(real_deck)} cards")
print(f"Sample cards: {real_deck[:10]}...")
print(f"\nMod Deck: {len(mod_deck)} cards")
for mod in mod_deck:
    print(f"  - {mod.name} ({mod.rarity}): {mod.description}")
print(f"\nPlayers: {len(players)} characters")
for player in players:
    print(f"  - {player.name} ({player.char_class}): {player.ability}")

Real Deck: 54 cards
Sample cards: ['2S', '2H', '2D', '2C', '3S', '3H', '3D', '3C', '4S', '4H']...

Mod Deck: 3 cards
  - Extra Draw (Rarity.Common): Draw 1 additional card
  - Sneaky Swap (Rarity.Uncommon): Swap a card with opponent
  - Royal Dividend (Rarity.Uncommon): Gain +3 chips

Players: 3 characters
  - Scruffy McMuffins (CharacterClass.Rogue): Steal 1 opponent card 2x per round
  - Patches (CharacterClass.Jester): Blind money refunded
  - Sir Whiskers (CharacterClass.Warrior): Reveal opponent card once per round


In [36]:
# Hand evaluation functions

def card_rank(card: CardValue) -> int:
    """Convert a card to its numeric rank."""
    if card == "JOKER":
        return 0
    
    rank_str = card[:-1]  # Everything except last character (suit)
    
    if rank_str in ["J", "Q", "K", "A"]:
        rank_map = {"J": 11, "Q": 12, "K": 13, "A": 14}
        return rank_map[rank_str]
    
    return int(rank_str)

def evaluate_standard_poker(hand: List[CardValue]) -> HandType:
    """Evaluate standard poker hands with joker support."""
    if len(hand) != 5:
        raise ValueError(f"Hand must contain exactly 5 cards, got {len(hand)}")
    
    jokers = [card for card in hand if card == "JOKER"]
    real_cards = [card for card in hand if card != "JOKER"]
    num_jokers = len(jokers)
    
    if num_jokers == 0:
        # No jokers - standard evaluation
        suits = [card[-1] for card in hand]
        ranks = [card_rank(card) for card in hand]
        rank_counts = {}
        
        for r in ranks:
            rank_counts[r] = rank_counts.get(r, 0) + 1
        
        sorted_ranks = sorted(ranks)
        is_flush = len(set(suits)) == 1
        is_straight = all(sorted_ranks[i+1] - sorted_ranks[i] == 1 for i in range(4))
        
        # Check hands in order of strength
        if is_straight and is_flush:
            if sorted_ranks == [10, 11, 12, 13, 14]:
                return cast(HandType, "Royal Flush")
            return cast(HandType, "Straight Flush")
        
        if 4 in rank_counts.values():
            return cast(HandType, "Four of a Kind")
        
        if sorted(rank_counts.values()) == [2, 3]:
            return cast(HandType, "Full House")
        
        if is_flush:
            return cast(HandType, "Flush")
        
        if is_straight:
            return cast(HandType, "Straight")
        
        if 3 in rank_counts.values():
            return cast(HandType, "Three of a Kind")
        
        pair_count = sum(1 for count in rank_counts.values() if count == 2)
        if pair_count == 2:
            return cast(HandType, "Two Pair")
        elif pair_count == 1:
            return cast(HandType, "One Pair")
        
        return cast(HandType, "High Card")
    else:
        # With jokers - simplified evaluation
        ranks = [card_rank(card) for card in real_cards]
        rank_counts = {}
        for r in ranks:
            rank_counts[r] = rank_counts.get(r, 0) + 1
        
        # Use jokers to make best possible hand
        for rank in set(ranks):
            if rank_counts[rank] + num_jokers >= 4:
                return cast(HandType, "Four of a Kind")
        
        for rank in set(ranks):
            if rank_counts[rank] + num_jokers >= 3:
                return cast(HandType, "Three of a Kind")
        
        for rank in set(ranks):
            if rank_counts[rank] + num_jokers >= 2:
                return cast(HandType, "One Pair")
        
        return cast(HandType, "High Card")

# Hand ranking order
HAND_RANK_ORDER = [
    "High Card", "One Pair", "Two Pair", "Three of a Kind", "Straight", 
    "Flush", "Full House", "Four of a Kind", "Straight Flush", "Royal Flush",
    "Flush Four", "Sandwich Hand", "Odd Straight", "Even Straight", 
    "Skipping Straight", "Rainbow Straight", "Flush House", "Five of a Kind", "Flush Five"
]

def get_hand_rank(hand_type: HandType) -> int:
    """Get numeric rank for comparison."""
    return HAND_RANK_ORDER.index(hand_type)

print("Hand evaluation functions loaded!")
print(f"Hand rankings (weakest to strongest): {HAND_RANK_ORDER[:5]}...")

Hand evaluation functions loaded!
Hand rankings (weakest to strongest): ['High Card', 'One Pair', 'Two Pair', 'Three of a Kind', 'Straight']...


In [37]:
# Main simulation function

def simulate_one_round() -> Tuple[str, HandType, int]:
    """
    Run one complete round of the poker simulation.
    
    Returns:
        Tuple of (winner_name, winning_hand_type, points_earned)
    """
    print("STARTING ROUND SIMULATION")
    print("=" * 50)
    
    # Initialize game state
    current_real_deck = real_deck.copy()
    random.shuffle(current_real_deck)
    current_mod_deck = mod_deck.copy()
    random.shuffle(current_mod_deck)
    
    # Track player data
    player_hands: Dict[str, List[CardValue]] = {}
    player_mods: Dict[str, List[ModCard]] = {}
    
    print("\nPHASE 1: DEALING HANDS")
    print("-" * 30)
    
    # Deal cards to each player
    for player in players:
        # Deal real cards (5-8 cards)
        num_real = min(random.randint(5, 8), len(current_real_deck))
        real_cards = [current_real_deck.pop() for _ in range(num_real)]
        
        # Deal mod cards to fill hand
        hand_size = 8
        num_mod = max(0, hand_size - len(real_cards))
        num_mod = min(num_mod, len(current_mod_deck))
        mod_cards = [current_mod_deck.pop() for _ in range(num_mod)] if num_mod > 0 else []
        
        player_hands[player.name] = real_cards
        player_mods[player.name] = mod_cards
        
        print(f"{player.name} ({player.char_class}):")
        print(f"  Real cards: {', '.join(real_cards[:5])}{'...' if len(real_cards) > 5 else ''}")
        print(f"  Mod cards: {[mod.name for mod in mod_cards]}")
    
    print("\nPHASE 2: APPLYING MOD EFFECTS")
    print("-" * 30)
    
    # Apply mod effects
    for player in players:
        for mod in player_mods[player.name]:
            if mod.effect == ModCardEffects.Draw1 and current_real_deck:
                drawn = current_real_deck.pop()
                player_hands[player.name].append(drawn)
                print(f"  -> {player.name} draws {drawn} (Extra Draw)")
            elif mod.effect == ModCardEffects.SneakySwap:
                # Simplified swap
                other_players = [p for p in players if p.name != player.name]
                if other_players and player_hands[player.name]:
                    target = random.choice(other_players)
                    if player_hands[target.name]:
                        my_card = player_hands[player.name].pop()
                        their_card = player_hands[target.name].pop()
                        player_hands[player.name].append(their_card)
                        player_hands[target.name].append(my_card)
                        print(f"  -> {player.name} swaps {my_card} <-> {their_card} with {target.name}")
    
    print("\nPHASE 3: CHARACTER ABILITIES")
    print("-" * 30)
    
    # Apply character abilities
    for player in players:
        if player.char_class == CharacterClass.Rogue:
            # Steal up to 2 cards
            for steal_num in range(2):
                targets = [p for p in players if p.name != player.name and player_hands[p.name]]
                if targets:
                    target = random.choice(targets)
                    stolen_card = player_hands[target.name].pop()
                    player_hands[player.name].append(stolen_card)
                    print(f"  -> {player.name} (Rogue) steals {stolen_card} from {target.name}")
        elif player.char_class == CharacterClass.Warrior:
            # Reveal one opponent card
            targets = [p for p in players if p.name != player.name and player_hands[p.name]]
            if targets:
                target = random.choice(targets)
                revealed = random.choice(player_hands[target.name])
                print(f"  -> {player.name} (Warrior) reveals {target.name}'s {revealed}")
    
    print("\nPHASE 4: HAND EVALUATION")
    print("-" * 30)
    
    # Evaluate best hands
    player_best_hands: Dict[str, HandType] = {}
    for player in players:
        cards = player_hands[player.name]
        best_hand = cast(HandType, "High Card")
        
        if len(cards) >= 5:
            # Try all 5-card combinations
            for combo in combinations(cards, 5):
                hand_type = evaluate_standard_poker(list(combo))
                if get_hand_rank(hand_type) > get_hand_rank(best_hand):
                    best_hand = hand_type
        
        player_best_hands[player.name] = best_hand
        print(f"  {player.name}: {best_hand}")
    
    # Determine winner
    winner = max(player_best_hands, key=lambda x: get_hand_rank(player_best_hands[x]))
    winner_hand = player_best_hands[winner]
    
    print(f"\nWINNER: {winner} with {winner_hand}!")
    
    # Calculate points
    high_tier_hands = {"Flush Five", "Five of a Kind"}
    points_earned = 2 if winner_hand in high_tier_hands else 1
    chips_earned = 0
    
    for mod in player_mods[winner]:
        if mod.effect == ModCardEffects.RoyalDividend:
            chips_earned += 3
            print(f"  {winner} gains +3 chips from {mod.name}")
    
    print(f"\nROUND RESULTS:")
    print(f"  Winner: {winner} (+{points_earned} points, +{chips_earned} bonus chips)")
    print(f"  Winning Hand: {winner_hand}")
    
    return winner, winner_hand, points_earned

print("Simulation function ready!")
print("Run the next cell to simulate a poker round!")

Simulation function ready!
Run the next cell to simulate a poker round!


In [ ]:
# Run multiple rounds for a tournament

def run_tournament(num_rounds: int = 10) -> Dict[str, int]:
    """Run a multi-round tournament."""
    print(f"STARTING {num_rounds}-ROUND TOURNAMENT")
    print("=" * 60)
    
    # Track player statistics
    player_stats = {player.name: {"wins": 0, "points": 0, "hands": []} for player in players}
    
    # Run all rounds
    for round_num in range(1, num_rounds + 1):
        print(f"\nROUND {round_num}/{num_rounds}")
        print("-" * 40)
        
        winner, hand_type, points = simulate_one_round()
        
        # Update statistics
        player_stats[winner]["wins"] += 1
        player_stats[winner]["points"] += points
        player_stats[winner]["hands"].append(hand_type)
        
        print(f"Round {round_num} Winner: {winner} ({hand_type})")
    
    print("\nTOURNAMENT RESULTS")
    print("=" * 60)
    
    # Sort players by points
    ranked_players = sorted(player_stats.items(), key=lambda x: x[1]["points"], reverse=True)
    
    # Display final standings
    medals = ["1st", "2nd", "3rd"]
    for rank, (name, stats) in enumerate(ranked_players):
        medal = medals[rank] if rank < 3 else f"{rank+1}th"
        print(f"{medal} {name}")
        print(f"    Points: {stats['points']} | Wins: {stats['wins']}/{num_rounds}")
        print(f"    Hands: {', '.join(stats['hands'])}")
        print()
    
    champion = ranked_players[0][0]
    print(f"TOURNAMENT CHAMPION: {champion}!")
    
    return {name: stats["points"] for name, stats in player_stats.items()}

# Run a 10-round tournament
print("Running a 10 round tournament...\n")
final_scores = run_tournament(10)

print(f"\nFinal Scores: {final_scores}")

Running 3-round tournament...

STARTING 3-ROUND TOURNAMENT

ROUND 1/3
----------------------------------------
STARTING ROUND SIMULATION

PHASE 1: DEALING HANDS
------------------------------
Scruffy McMuffins (CharacterClass.Rogue):
  Real cards: 7H, 2H, 4H, 2C, 7D
  Mod cards: ['Extra Draw', 'Sneaky Swap', 'Royal Dividend']
Patches (CharacterClass.Jester):
  Real cards: QC, 8D, 7S, 5H, 4D...
  Mod cards: []
Sir Whiskers (CharacterClass.Warrior):
  Real cards: JOKER, 9H, 10D, JH, QD...
  Mod cards: []

PHASE 2: APPLYING MOD EFFECTS
------------------------------
  -> Scruffy McMuffins draws QH (Extra Draw)
  -> Scruffy McMuffins swaps QH <-> 10H with Sir Whiskers

PHASE 3: CHARACTER ABILITIES
------------------------------
  -> Scruffy McMuffins (Rogue) steals QH from Sir Whiskers
  -> Scruffy McMuffins (Rogue) steals 10S from Patches
  -> Sir Whiskers (Warrior) reveals Patches's 7S

PHASE 4: HAND EVALUATION
------------------------------
  Scruffy McMuffins: Flush
  Patches: High Car

In [30]:
# Import visualization libraries
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

print("Visualization libraries loaded!")
print("Available chart types: Bar, Pie, Scatter, Heatmap, and more!")

Visualization libraries loaded!
Available chart types: Bar, Pie, Scatter, Heatmap, and more!


In [31]:
# Visualization functions
def create_tournament_dashboard(tournament_data: Dict):
    """Create a comprehensive tournament dashboard with multiple visualizations."""
    
    player_stats = tournament_data["player_stats"]
    round_results = tournament_data["round_results"]
    hand_frequency = tournament_data["hand_frequency"]
    num_rounds = tournament_data["num_rounds"]
    
    # Prepare data for visualizations
    player_names = list(player_stats.keys())
    player_points = [player_stats[name]["points"] for name in player_names]
    player_wins = [player_stats[name]["wins"] for name in player_names]
    player_classes = [player_stats[name]["character_class"] for name in player_names]
    
    # Create subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            "Final Tournament Standings", 
            "Hand Type Distribution",
            "Wins by Character Class",
            "Round-by-Round Performance"
        ),
        specs=[[{"type": "bar"}, {"type": "pie"}],
               [{"type": "bar"}, {"type": "scatter"}]]
    )
    
    # 1. Tournament Standings (Bar Chart)
    fig.add_trace(
        go.Bar(
            x=player_names,
            y=player_points,
            name="Points",
            text=[f"{pts} pts<br>{wins} wins" for pts, wins in zip(player_points, player_wins)],
            textposition="auto",
            marker_color=px.colors.qualitative.Set3[:len(player_names)]
        ),
        row=1, col=1
    )
    
    # 2. Hand Distribution (Pie Chart)
    fig.add_trace(
        go.Pie(
            labels=list(hand_frequency.keys()),
            values=list(hand_frequency.values()),
            name="Hand Types"
        ),
        row=1, col=2
    )
    
    # 3. Wins by Character Class (Bar Chart)
    class_wins = {}
    for name in player_names:
        char_class = player_stats[name]["character_class"]
        wins = player_stats[name]["wins"]
        class_wins[char_class] = class_wins.get(char_class, 0) + wins
    
    fig.add_trace(
        go.Bar(
            x=list(class_wins.keys()),
            y=list(class_wins.values()),
            name="Class Wins",
            marker_color=px.colors.qualitative.Pastel1[:len(class_wins)]
        ),
        row=2, col=1
    )
    
    # 4. Round Performance (Scatter Plot)
    for i, name in enumerate(player_names):
        rounds = list(range(1, num_rounds + 1))
        cumulative_points = []
        points_so_far = 0
        
        for round_result in round_results:
            if round_result["winner"] == name:
                points_so_far += round_result["points"]
            cumulative_points.append(points_so_far)
        
        fig.add_trace(
            go.Scatter(
                x=rounds,
                y=cumulative_points,
                mode='lines+markers',
                name=f"{name}",
                line=dict(width=3),
                marker=dict(size=8)
            ),
            row=2, col=2
        )
    
    # Update layout
    fig.update_layout(
        height=800,
        title_text=f"🎲 Poker Tournament Dashboard - {num_rounds} Rounds",
        title_x=0.5,
        showlegend=True,
        template="plotly_white"
    )
    
    # Update axes labels
    fig.update_xaxes(title_text="Players", row=1, col=1)
    fig.update_yaxes(title_text="Points", row=1, col=1)
    fig.update_xaxes(title_text="Character Class", row=2, col=1)
    fig.update_yaxes(title_text="Total Wins", row=2, col=1)
    fig.update_xaxes(title_text="Round Number", row=2, col=2)
    fig.update_yaxes(title_text="Cumulative Points", row=2, col=2)
    
    return fig

def create_character_analysis(tournament_data: Dict):
    """Create detailed character class performance analysis."""
    player_stats = tournament_data["player_stats"]
    
    # Analyze performance by character class
    class_analysis = {}
    for name, stats in player_stats.items():
        char_class = stats["character_class"]
        if char_class not in class_analysis:
            class_analysis[char_class] = {
                "players": [],
                "total_points": 0,
                "total_wins": 0,
                "hands_won_with": []
            }
        
        class_analysis[char_class]["players"].append(name)
        class_analysis[char_class]["total_points"] += stats["points"]
        class_analysis[char_class]["total_wins"] += stats["wins"]
        class_analysis[char_class]["hands_won_with"].extend(stats["hands"])
    
    # Create visualization
    classes = list(class_analysis.keys())
    avg_points = [class_analysis[cls]["total_points"] / len(class_analysis[cls]["players"]) 
                  for cls in classes]
    total_wins = [class_analysis[cls]["total_wins"] for cls in classes]
    
    fig = go.Figure()
    
    # Add bars for average points
    fig.add_trace(
        go.Bar(
            name="Avg Points per Player",
            x=classes,
            y=avg_points,
            yaxis="y",
            offsetgroup=1,
            marker_color=px.colors.qualitative.Set2
        )
    )
    
    # Add bars for total wins
    fig.add_trace(
        go.Bar(
            name="Total Wins",
            x=classes,
            y=total_wins,
            yaxis="y2",
            offsetgroup=2,
            marker_color=px.colors.qualitative.Dark2
        )
    )
    
    fig.update_layout(
        title="⚔️ Character Class Performance Analysis",
        xaxis=dict(title="Character Class"),
        yaxis=dict(title="Average Points per Player", side="left"),
        yaxis2=dict(title="Total Wins", side="right", overlaying="y"),
        template="plotly_white",
        height=500
    )
    
    return fig

print("Visualization functions created!")
print("Ready to generate beautiful charts from tournament data!")

Visualization functions created!
Ready to generate beautiful charts from tournament data!


In [38]:
# Enhanced tournament function with detailed tracking
def run_tournament_with_stats(num_rounds: int = 5) -> Dict:
    """Run a tournament with comprehensive statistics tracking."""
    print(f"STARTING {num_rounds}-ROUND TOURNAMENT WITH VISUALIZATION")
    print("=" * 60)
    
    # Initialize comprehensive tracking
    player_stats = {
        player.name: {
            "wins": 0, 
            "points": 0, 
            "hands": [], 
            "character_class": player.char_class.value,
            "abilities_used": 0,
            "mod_cards_played": 0,
            "round_placements": []
        } 
        for player in players
    }
    
    round_results = []
    hand_frequency = {}
    
    # Run all rounds
    for round_num in range(1, num_rounds + 1):
        print(f"\nROUND {round_num}/{num_rounds}")
        print("-" * 40)
        
        winner, hand_type, points = simulate_one_round()
        
        # Track hand frequency
        hand_frequency[hand_type] = hand_frequency.get(hand_type, 0) + 1
        
        # Update player statistics
        player_stats[winner]["wins"] += 1
        player_stats[winner]["points"] += points
        player_stats[winner]["hands"].append(hand_type)
        
        # Record round result
        round_results.append({
            "round": round_num,
            "winner": winner,
            "hand_type": hand_type,
            "points": points
        })
        
        # Track placements for all players this round
        round_scores = {}
        for player_name in player_stats.keys():
            if player_name == winner:
                round_scores[player_name] = get_hand_rank(hand_type)
            else:
                # Simulate other players' hands for placement
                round_scores[player_name] = random.randint(0, get_hand_rank(hand_type) - 1)
        
        # Sort by score and assign placements
        sorted_players = sorted(round_scores.items(), key=lambda x: x[1], reverse=True)
        for placement, (player_name, _) in enumerate(sorted_players, 1):
            player_stats[player_name]["round_placements"].append(placement)
        
        print(f"Round {round_num} Winner: {winner} ({hand_type})")
    
    print("\nTOURNAMENT COMPLETED - GENERATING VISUALIZATIONS...")
    
    return {
        "player_stats": player_stats,
        "round_results": round_results,
        "hand_frequency": hand_frequency,
        "num_rounds": num_rounds
    }

print("Enhanced tournament function ready!")
print("This version tracks detailed statistics for rich visualizations.")

Enhanced tournament function ready!
This version tracks detailed statistics for rich visualizations.


In [39]:
# Run enhanced tournament with visualizations!

print("🎲 RUNNING ENHANCED TOURNAMENT WITH VISUALIZATIONS")
print("=" * 60)

# Run tournament with detailed tracking
tournament_data = run_tournament_with_stats(5)

print(f"\n📊 GENERATING INTERACTIVE DASHBOARDS...")
print("-" * 40)

# Create and display main dashboard
dashboard = create_tournament_dashboard(tournament_data)
dashboard.show()

print("✅ Main tournament dashboard displayed above!")

🎲 RUNNING ENHANCED TOURNAMENT WITH VISUALIZATIONS
STARTING 5-ROUND TOURNAMENT WITH VISUALIZATION

ROUND 1/5
----------------------------------------
STARTING ROUND SIMULATION

PHASE 1: DEALING HANDS
------------------------------
Scruffy McMuffins (CharacterClass.Rogue):
  Real cards: 8C, 9S, 2D, 9D, 2S
  Mod cards: ['Royal Dividend', 'Extra Draw', 'Sneaky Swap']
Patches (CharacterClass.Jester):
  Real cards: 10C, 5S, 4H, 2H, 3C
  Mod cards: []
Sir Whiskers (CharacterClass.Warrior):
  Real cards: 5C, 2C, 3S, 3D, 5H...
  Mod cards: []

PHASE 2: APPLYING MOD EFFECTS
------------------------------
  -> Scruffy McMuffins draws 9H (Extra Draw)
  -> Scruffy McMuffins swaps 9H <-> JS with Sir Whiskers

PHASE 3: CHARACTER ABILITIES
------------------------------
  -> Scruffy McMuffins (Rogue) steals 9H from Sir Whiskers
  -> Scruffy McMuffins (Rogue) steals 8H from Sir Whiskers
  -> Sir Whiskers (Warrior) reveals Patches's 5S

PHASE 4: HAND EVALUATION
------------------------------
  Scruffy M

✅ Main tournament dashboard displayed above!


In [40]:
# Hand rarity and exotic hand analysis

def create_hand_rarity_analysis(tournament_data: Dict):
    """Analyze the rarity and frequency of different hand types."""
    hand_frequency = tournament_data["hand_frequency"]
    
    # Categorize hands by rarity
    common_hands = ["High Card", "One Pair", "Two Pair"]
    uncommon_hands = ["Three of a Kind", "Straight", "Flush"]
    rare_hands = ["Full House", "Four of a Kind", "Straight Flush"]
    exotic_hands = ["Royal Flush", "Flush Four", "Sandwich Hand", "Odd Straight", 
                   "Even Straight", "Skipping Straight", "Rainbow Straight", 
                   "Flush House", "Five of a Kind", "Flush Five"]
    
    categories = {
        "Common": sum(hand_frequency.get(hand, 0) for hand in common_hands),
        "Uncommon": sum(hand_frequency.get(hand, 0) for hand in uncommon_hands),
        "Rare": sum(hand_frequency.get(hand, 0) for hand in rare_hands),
        "Exotic": sum(hand_frequency.get(hand, 0) for hand in exotic_hands)
    }
    
    # Create donut chart
    fig = go.Figure(data=[go.Pie(
        labels=list(categories.keys()),
        values=list(categories.values()),
        hole=0.4,
        marker_colors=px.colors.qualitative.Set3
    )])
    
    fig.update_layout(
        title="🃏 Hand Rarity Distribution",
        template="plotly_white",
        height=400
    )
    
    # Add center text
    total_hands = sum(categories.values())
    exotic_pct = (categories["Exotic"] / total_hands * 100) if total_hands > 0 else 0
    
    fig.add_annotation(
        text=f"Exotic Hands<br>{exotic_pct:.1f}%",
        x=0.5, y=0.5,
        font_size=16,
        showarrow=False
    )
    
    return fig

# Create and display hand rarity analysis
print("🃏 ANALYZING HAND RARITY DISTRIBUTION")
print("-" * 40)

hand_rarity_chart = create_hand_rarity_analysis(tournament_data)
hand_rarity_chart.show()

print("✅ Hand rarity analysis displayed above!")

# Show detailed breakdown
print("\nDetailed Hand Frequency:")
for hand_type, count in sorted(tournament_data["hand_frequency"].items(), 
                              key=lambda x: x[1], reverse=True):
    percentage = (count / tournament_data["num_rounds"]) * 100
    print(f"  {hand_type}: {count} times ({percentage:.1f}%)")

🃏 ANALYZING HAND RARITY DISTRIBUTION
----------------------------------------


✅ Hand rarity analysis displayed above!

Detailed Hand Frequency:
  Full House: 2 times (40.0%)
  One Pair: 1 times (20.0%)
  Two Pair: 1 times (20.0%)
  Three of a Kind: 1 times (20.0%)
